# CART Amélioré

L'objectif de ce notebook est d'utiliser le même modèle d'arbre de régression, mais d'y ajouter des modifications supplémentaires, notamment les dates par semaine ainsi que d'autres variables.

In [3]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

In [4]:
prices = pd.read_csv("../data/price_availability.csv", sep=";")
listings = pd.read_csv("../clean/listings_final.csv", sep=";")
listings = listings.drop(589)  # Cet individu n'a pas de prix !
print("Chargement des données effectué.")

Chargement des données effectué.


In [5]:
uniques = np.unique(prices["listing_id"]) 
intersects = np.intersect1d(uniques, np.unique(listings["listing_id"]))
listings_prices = prices[prices["listing_id"].isin(intersects)]

In [6]:
dates =  pd.to_datetime(listings_prices["day"], format='%Y-%m-%d')
# TODO : Regrouper les prix par plages de 7 jours

# END TODO

,listing_id,date,local_price
0,56093,2018-08-27,170.0
1,56093,2018-09-03,170.0
2,56093,2018-09-10,170.0
3,56093,2018-09-17,170.0
4,56093,2018-09-24,170.0


In [7]:
week_prices.shape

(18981, 3)

In [8]:
week_prices.head()

,listing_id,date,local_price
0,56093,2018-08-27,170.0
1,56093,2018-09-03,170.0
2,56093,2018-09-10,170.0
3,56093,2018-09-17,170.0
4,56093,2018-09-24,170.0


On considère dorénavant un jeu de données $X$ où chaque individu, toujours identifié par le *listing_id*, est la fusion entre les variables de *listings* et le couple date/prix de *week_prices*. Cette fusion peut s'effectuer par le code suivant :

In [9]:
def populate(row):
    return row.append(listings[listings["listing_id"] == row["listing_id"]].iloc[0,2:])
    
X = week_prices.apply(populate, axis=1)

On vérifie que l'on obtient bien le même nombre d'individus.

In [10]:
X.shape

(18981, 20)

In [11]:
X.columns

Index(['listing_id', 'date', 'local_price', 'name', 'type', 'city',
       'neighborhood', 'latitude', 'longitude', 'person_capacity', 'beds',
       'bedrooms', 'bathrooms', 'is_rebookable', 'is_new_listing',
       'is_fully_refundable', 'is_host_highly_rated',
       'is_business_travel_ready', 'pricing_weekly_factor',
       'pricing_monthly_factor'],
      dtype='object')

Maintenant, on encode numériquement deux variables : les dates et le type.

In [25]:
label_encoder = LabelEncoder()
X["label_dates"] = # TODO
X["label_type"] = # TODO

On sépare le jeu de données entre un ensemble d'entraînement ($80 \%$) et un ensemble de test ($20 \%$).

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X.loc[:, ["label_dates", "label_type", "latitude", "longitude", "person_capacity", "bedrooms", "bathrooms"]],
    X.loc[:, "local_price"],
    test_size=0.2)

Il ne reste plus qu'à créer l'arbre de régression et de l'optimiser par rapport à nos nouvelles données.

In [43]:
# TODO : Créer l'arbre de régression.

# END TODO

DecisionTreeRegressor(criterion='mse', max_depth=8, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [44]:
print('Variance score: %.4f' % r2_score(Y_test, air_bnb_tree.predict(X_test)))

Variance score: 0.6877


In [45]:
import graphviz 
dot_data = tree.export_graphviz(air_bnb_tree, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("air_bnb_tree")

'air_bnb_tree.pdf'